### 用于特征提取和图像匹配变换

In [1]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [2]:
# 1. 特征提取
def extract_features(image):
    # image_array = np.array(image)
    feature_extractor = cv2.SIFT_create()
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# 2. 特征匹配
def match_features_bf(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

def match_features_flann(descriptors1, descriptors2):
    # FLANN 参数设置
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    # 创建 FLANN 匹配器
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # 使用 KNN 进行匹配
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    # 选择好的匹配
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    return good_matches
# 注掉的方法是基于透视变换
# # 3. 变换估计
# def estimate_transform(keypoints1, keypoints2, good_matches):
#     src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     # 使用RANSAC算法估计变换矩阵
#     M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
#     return M

# #4. 变换应用
# def apply_transform(image, M):
#     # 应用估计的变换矩阵到图像
#     transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
#     return transformed_image

# 3. 仿射变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计仿射变换矩阵
    M = cv2.estimateAffinePartial2D(src_pts, dst_pts)[0]
    return M

# 4. 仿射变换应用
def apply_transform(image, M):
    
    # 应用估计的仿射变换矩阵到图像
    # image = Image.fromarray(image)
    transformed_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return transformed_image

In [5]:
# 一次配准
image1 = cv2.imread(r'origin_images_2nd\B0.bmp', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
image2 = cv2.imread(r'origin_images_2nd\B5.bmp', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
# array = np.load('anti_mean.npy')
# image1 = np.clip(image1 / array,0,255)
# image2 = np.clip(image2 / array,0,255)
# image1 = image1.astype(np.uint8)
# image2 = image2.astype(np.uint8)
assert image1.shape == image2.shape

In [5]:
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)
# 2. 特征匹配
good_matches = match_features_bf(descriptors1, descriptors2)
print(len(good_matches))

0


In [4]:
# 3. 变换估计
M = estimate_transform(keypoints1, keypoints2, good_matches)
# 4. 变换应用
transformed_image = apply_transform(image1, M)
cv2.imwrite('trans_image/transformed2to222.png', transformed_image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\ptsetreg.cpp:176: error: (-215:Assertion failed) count >= 0 && count2 == count in function 'cv::RANSACPointSetRegistrator::run'


### 后面是SIFT和BFMatch合理性判断

In [3]:
#一次配准
image1 = cv2.imread(r'image_test\a.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
image2 = cv2.imread(r'image_test\b.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)

AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)
matches = cv2.BFMatcher.knnMatch(descriptors1,descriptors2, k=2)
good_matches = match_features_bf(descriptors1, descriptors2)
img5 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,matches,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
pillow_image = Image.fromarray(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [ ]:
img6 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,good,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
pillow_image = Image.fromarray(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
pillow_image.show()